# Breast Cancer Classifier Comparison

By Iván del Horno Sotillo

This notebook is part of a Bachelor Thesis project. The main goal of this project is to compare different machine learning algorithms in order to find the best one for the classification of breast cancer. The dataset used is the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) from the UCI Machine Learning Repository.

The dataset contains 569 samples of benign and malignant tumor cells with features computed from an image obtained from a FNAB (Fine Needle Aspirate Biopsy).

More information will be available in the Thesis document.

## 1. Preparation

### Importing libraries and dataset

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer

In [2]:
# Load the data
data = load_breast_cancer()

### Basic information about the dataset

In [16]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

Thanks to importing the dataset directly from Scikit-Learn, we can get some basic information about it.
We can see that we have 569 samples, out of which 212 are malignant and 357 are benign. The dataset has 30 features which are all numeric values. We can also see that there are no missing values.

The 30 features are based on 10 attributes (listed in the description above), and from each attribute we obtain the mean, the standard error and the worst (mean of the three largest values).

In [17]:
print(data.feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
